# Quantified McRae norms

In [13]:
from nltk import agreement
from calculate_iaa import proportional_agreement_pairs
#from calculate_iaa import get_average_kappa
from calculate_iaa import coder_pairs_matrix
from sklearn.metrics import cohen_kappa_score
import numpy as np

from collections import defaultdict

def load_mcrae():
    matrix = []
    c = 0
    with open('../data/mcrae-quantified.txt') as infile:
        for line in infile:
            if not line.startswith('#') and '\t' in line:
                c += 1
                row = line.strip().split('\t')
                pair = row[2]+'-'+row[1]
                #print(line.strip())
                annotations = row[3:6]
                for n, annotation in enumerate(annotations):
                    #row = [worker, quid, answer]
                    if n == 3:
                        print(n, pair, annotation)
                    matrix_row = [n, pair, annotation]
                    matrix.append(matrix_row)
    
    return matrix  

matrix = load_mcrae()
print(len(matrix))
ratingtask = agreement.AnnotationTask(data=matrix)
alpha = ratingtask.alpha()
kappa = get_average_kappa(matrix)
prop = proportional_agreement_pairs(matrix)
print(alpha, kappa,  prop)

21771
0.27469428135421126 0.2963650774961838 0.5257911901153178


In [12]:
pair_kappa = get_average_kappa_pairs(matrix)
for pair, k in pair_kappa.items():
    print(pair, k)

(0, 1) 0.27857131002219504
(0, 2) 0.23075919837842918
(1, 2) 0.37976472408792716


In [11]:
def get_average_kappa(matrix):
    pair_kappa_dict = get_kappa_pairs(matrix)
    
    if len(pair_kappa_dict) > 0:
        sum_kappa = sum(pair_kappa_dict.values())
        av_kappa = sum_kappa/len(pair_kappa_dict)
    else:
        av_kappa = 0
    return av_kappa
   

def get_kappa_pairs(matrix):
    pairs = coder_pairs_matrix(matrix)
    unit_dict = defaultdict(dict)
    pair_unit_dict = defaultdict(list)
    pair_kappa_dict = dict()
    for w, u, l in matrix:
        unit_dict[u][w] = l
    all_pair_answers = []
    sum_kappas =0.0
    sum_valid_pairs = 0.0
    for wi, wj in pairs:
        pair_label_dict = defaultdict(list)
        for u, worker_l_dict in unit_dict.items():
            if wi in worker_l_dict and wj in worker_l_dict:
                pair_label_dict[wi].append(worker_l_dict[wi])
                pair_label_dict[wj].append(worker_l_dict[wj])
        labels_i = pair_label_dict[wi]
        labels_j = pair_label_dict[wj]
        if len(labels_i) > 0:
            kappa = cohen_kappa_score(labels_i, labels_j)
            pair_kappa_dict[(wi, wj)] = kappa
            
    return pair_kappa_dict